# Convolutional Neural Networks

<img src="https://www.cntk.ai/jup/cntk103d_conv2d_final.gif" width=175px>


## Setup

Using the Keras library to import the [MNIST](http://yann.lecun.com/exdb/mnist/) dataset due to the simplicity over other libraries (e.x. [CNTK](https://cntk.ai/pythondocs/CNTK_103A_MNIST_DataLoader.html#) and [TensorFlow](https://www.tensorflow.org/get_started/mnist/pros))
<img src="https://upload.wikimedia.org/wikipedia/commons/2/27/MnistExamples.png", width=400px>

In [1]:
import keras
from keras import backend as K

# Loading the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()

# Input image dimensions
img_rows, img_cols = 28, 28

# Ensuring the channels are in the correct 
if K.image_data_format() == 'channels_first':
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
    
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'Training samples')
print(X_test.shape[0], 'Testing samples')

Using TensorFlow backend.


X_train shape: (60000, 28, 28, 1)
60000 Training samples
10000 Testing samples


## Keras

In [2]:
import sys
import time
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import advanced_activations
from keras.layers.normalization import BatchNormalization
from keras import backend as K

print(time.strftime('%Y/%m/%d %H:%M'))
print('OS: ', sys.platform)
print('Python: ', sys.version)
print('NumPy: ', np.__version__)
print('Keras: ', keras.__version__)

# Printing backend and GPU information
if keras.backend.backend() == 'tensorflow':
    import tensorflow as tf
    from tensorflow.python.client import device_lib
    print('Backend: TensorFlow', tf.__version__)
    local_device_protos = device_lib.list_local_devices()
    print([x for x in local_device_protos if x.device_type == 'GPU'])

    # Avoiding memory issues with the GPU
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    sess = tf.Session(config=config)
    K.set_session(sess)

elif keras.backend.backend() == 'CNTK':
    import cntk as C
    print('Backend: CNTK', C.__version__)
    print('GPU: ', c.gpu(0))


# For tracking the training time for each epoch
from keras import callbacks
import time

class TimeHistory(callbacks.Callback):
    """
    Tracks training time on individual epochs for a Keras model
    """
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)


time_callback = TimeHistory()

# Setting data types and normalizing the images
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'Training samples')
print(X_test.shape[0], 'Testing samples')

# Model settings
batch_size = 128
num_classes = 10
epochs = 12

# Converting class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


# Beginning model building
model = Sequential()

# Layer 1 - Conv (5x5)
model.add(Conv2D(32, kernel_size=(5, 5), input_shape=input_shape))
model.add(advanced_activations.LeakyReLU(alpha=0.03))

# Layer 2 - Conv (5x5) & Max Pooling
model.add(Conv2D(32, kernel_size=(5, 5)))
model.add(advanced_activations.LeakyReLU(alpha=0.03))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

# Layer 3 - Conv (3x3)
model.add(Conv2D(64, kernel_size=(3, 3)))
model.add(advanced_activations.LeakyReLU(alpha=0.03))

# Layer 4 - Conv (3x3) & Max Pooling
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(advanced_activations.LeakyReLU(alpha=0.03))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

# Layer 5 - FC 1024
model.add(Flatten())
model.add(Dense(1024, activation='relu'))

# Layer 6 - FC 1024
model.add(Dense(1024, activation='relu'))

# Output Layer
model.add(Dense(num_classes, activation='softmax'))

# Defining loss function, optimizer, and metrics to report
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

# Summary of the model before fitting
model.summary()

# Fitting the model
model.fit(X_train, y_train,
          epochs=epochs,
          batch_size=128, verbose=1,
          validation_split=0.1,  # Uses last 10% of data (not shuffled) for validation
          callbacks=[time_callback])  # Gives epoch training times

# Getting test information
score, acc = model.evaluate(X_test, y_test,
                            batch_size=batch_size)
print()
print('Test score:', score)
print('Test accuracy:', acc)

# Saving the model
# model.save('model.h5')

# Reporting total training time
total_training_time = round(sum(time_callback.times))
print('Total training time for {0} epochs: {1} seconds'.format(epochs, total_training_time))

2018/01/18 21:16
OS:  win32
Python:  3.6.2 |Anaconda custom (64-bit)| (default, Jul 20 2017, 12:30:02) [MSC v.1900 64 bit (AMD64)]
NumPy:  1.13.3
Keras:  2.0.6
Backend: TensorFlow 1.3.0
[name: "/gpu:0"
device_type: "GPU"
memory_limit: 1524796620
locality {
  bus_id: 1
}
incarnation: 18240986228833159361
physical_device_desc: "device: 0, name: GeForce GTX 680, pci bus id: 0000:01:00.0"
]
X_train shape: (60000, 28, 28, 1)
60000 Training samples
10000 Testing samples
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 24, 24, 32)        832       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 24, 24, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 20, 20, 32)        25632     
_________________________________________________________________
lea

### Additional Setup

Splitting the training dataset into training and validation since Keras automatically does this when fitting the model

In [2]:
from sklearn.model_selection import train_test_split
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=.2)

## CNTK

In [3]:
import sys
import time
import numpy as np
import cntk as C
from cntk.layers import Convolution2D, MaxPooling, Dropout, Dense

print(time.strftime('%Y/%m/%d %H:%M'))
print('OS: ', sys.platform)
print('Python: ', sys.version)
print('NumPy: ', np.__version__)
print('CNTK: ', C.__version__)
print('GPU: ', C.gpu(0))

2018/01/19 17:38
OS:  win32
Python:  3.6.2 |Anaconda custom (64-bit)| (default, Jul 20 2017, 12:30:02) [MSC v.1900 64 bit (AMD64)]
NumPy:  1.13.3
CNTK:  2.1
GPU:  GPU[0] GeForce GTX 680


Reference [1](https://github.com/Microsoft/CNTK/blob/master/Examples/Image/Classification/ConvNet/Python/ConvNet_MNIST.py) | [2](https://cntk.ai/pythondocs/CNTK_103D_MNIST_ConvolutionalNeuralNetwork.html) | [3](https://medium.com/@tuzzer/building-a-deep-handwritten-digits-classifier-using-microsoft-cognitive-toolkit-6ae966caec69)

In [26]:
# Define the data dimensions
input_dim_model = (1, 28, 28)  # Images are 28 x 28 with 1 channel of color (gray)
input_dim = 28*28  # used by readers to treat input data as a vector
num_output_classes = 10

X = C.input_variable(input_dim_model)
y = C.input_variable(num_output_classes)

def create_model(input_vars, out_dims=10, dropout_prob=0.0):
    with C.layers.default_options(init=C.glorot_uniform(), activation=C.relu):

        conv1 = Convolution2D(filter_shape=(5, 5),
                            num_filters=8,
                            strides=(2, 2),
                            pad=True,
                            init_bias=0.1,
                            name='conv1')(input_vars)
        pooling1 = MaxPooling((2, 2),
                              strides=(2, 2),
                              pad=True)(conv1)

        conv2 = Convolution2D(filter_shape=(5, 5),
                            num_filters=64,
                            strides=1,
                            pad=True,
                            init_bias=0.1,
                            name='conv2')(pooling1)
        pooling2 = MaxPooling((2, 2),
                              strides=(2, 2),
                              pad=True)(conv2)

        fc1 = Dense(1024,
                    init_bias=0.1,
                    name='fc1')(pooling2)
        
        dropout1 = Dropout(dropout_prob)(fc1)
        
        output_layer = Dense(out_dims,
                             activation=None,
                             init_bias=0.1,
                             name='classify')(dropout1)

        return output_layer
        
# Defining the input to the neural network
input_vars = C.input_variable(input_dim, np.float32)
        
# Create the model
output = create_model(X)

# Print the output shapes / parameters of different components
print("Output Shape of the first convolution layer:", output.conv1.shape)
print("Bias value of the last dense layer:", output.classify.b.value)

# Number of parameters in the network
C.logging.log_number_of_parameters(output)

### Setting up the trainer
# Define the label as the other input parameter of the trainer
labels = C.input_variable(num_output_classes, np.float32)

# Initialize the parameters for the trainer
train_minibatch_size = 50
# lr_schedule = C.learners.learning_parameter_schedule_per_sample(lr_per_sample, epoch_size=epoch_size)
momentum = 0.9

# Define the loss function
loss = C.cross_entropy_with_softmax(output, labels)

# Define the function that calculates classification error
label_error = C.classification_error(output, labels)

# Instantiate the trainer object to drive the model training
# learner = C.learners.momentum_sgd(output.parameters, lr_schedule, momentum)
# trainer = C.Trainer(output, loss, label_error, [learner])

Output Shape of the first convolution layer: (8, 14, 14)
Bias value of the last dense layer: [ 0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1]
Training 1072922 parameters in 8 parameter tensors.


## MXNet

## PyTorch

## TensorFlow

In [6]:
import sys
import time
import numpy as np
import tensorflow as tf

print(time.strftime('%Y/%m/%d %H:%M'))
print('OS: ', sys.platform)
print('Python: ', sys.version)
print('NumPy: ', np.__version__)
print('TensorFlow: ', tf.__version__)

# Checking tensorflow processing devices
from tensorflow.python.client import device_lib
local_device_protos = device_lib.list_local_devices()
print([x for x in local_device_protos if x.device_type == 'GPU'])

# Avoiding memory issues with the GPU
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)


2018/01/18 21:22
OS:  win32
Python:  3.6.2 |Anaconda custom (64-bit)| (default, Jul 20 2017, 12:30:02) [MSC v.1900 64 bit (AMD64)]
NumPy:  1.13.3
TensorFlow:  1.3.0
[name: "/gpu:0"
device_type: "GPU"
locality {
  bus_id: 1
}
incarnation: 3341888561636021422
physical_device_desc: "device: 0, name: GeForce GTX 680, pci bus id: 0000:01:00.0"
]
